In [3]:
import sys
sys.path.append("D:\毕业论文")
import ebooklib
from bs4 import BeautifulSoup
from ebooklib import epub
from deepseek import get_response,generate_prompt
import xml.etree.ElementTree as ET
from utils import extract_nav_structure, classify_chapter
from prompts import chapter_classifier_prompt
book = epub.read_epub("D:\毕业论文\data\十三经译注.epub")
nav_tree = extract_nav_structure(book)

c:\Users\qq527\anaconda3\envs\final_paper\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
c:\Users\qq527\anaconda3\envs\final_paper\lib\site-packages\ebooklib\epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [4]:
# print(nav_tree.children[1].children[1].children[8].content_src)


# print(nav_tree.children[1].children[8].get_content(book))
content = nav_tree.children[1].children[5].get_content(book)
print(content)


<h1 id="sigil_toc_id_3">庄公</h1>&#13;
<p class="kindle-cn-para-no-indent">【题解】</p>&#13;
<p>&#13;
<span style="font-size:0.8em;">鲁庄公名同，取名之由，是生日正巧与父亲桓公相同之故。桓公死时，子同才十三岁；由于死得意外，生前来不及考虑为儿子定亲的事。母亲文姜，又似乎把心思都花在与同父异母兄齐襄公私通上，竟把儿子的婚姻大事耽搁下来。于是鲁庄公成年以后便进行了一次自由恋爱，与任姓支族党氏之女孟任割臂为盟，私定终身。但这次自主婚姻在“奔则为妾”的时代里得不到正式承认。庄公二十一年，母文姜死；二十四年，才娶齐女哀姜为夫人，这年他已三十七岁了。这次正式婚姻并不中庄公的意，他的情爱仍在孟任身上。哀姜受到冷落之余，与庄公之弟庆父、叔牙私通，并造成了庄公身后“鲁难未已”的动乱局面。庄公在位三十二年（前693—前662年）。</span>&#13;
</p>&#13;
<p class="kindle-cn-para-no-indent">【原文】</p>&#13;
<p>&#13;
<span style="font-size:0.9em;">3.1.1　元年春王正月。</span>&#13;
</p>&#13;
<p>&#13;
<span style="font-size:1.2em;">公何以不言即位？《春秋》君弑，子不言即位。君弑则子何以不言即位？隐之也<span class="math-super">〔1〕</span>&#13;
 。孰隐？隐子也。</span>&#13;
</p>&#13;
<p class="kindle-cn-para-no-indent">【注释】</p>&#13;
<p>〔1〕隐：伤痛。《解诂》：“隐痛是子之祸，不忍言即位。”</p>&#13;
<p class="kindle-cn-para-no-indent">【译文】</p>&#13;
<p>&#13;
<span style="font-size:0.8em;">元年春周历正月。</span>&#13;
</p>&#13;
<p>&#13;
<span style="font-size:0.8em;">庄公为什么不说即位？《春秋》国君被杀，儿子不说即位。国君被杀那

In [5]:
import re
pattern = re.compile(r'<(\w+)([^>]*)>')

# 使用正则表达式查找所有匹配的标签
matches = pattern.findall(content)

# 统计标签及其出现次数
tag_count = {}
for match in matches:
    tag, attributes = match
    full_tag = f'<{tag}{attributes}>'
    if full_tag in tag_count:
        tag_count[full_tag] += 1
    else:
        tag_count[full_tag] = 1

# 输出统计结果
for tag, count in tag_count.items():
    if count > 1:
        print(f'{tag}: {count}次')

<p class="kindle-cn-para-no-indent">: 244次
<p>: 880次
<span style="font-size:0.8em;">: 271次
<span style="font-size:0.9em;">: 180次
<span style="font-size:1.2em;">: 91次
<span class="math-super">: 338次
<img alt="" class="kindle-cn-inline-character3" src="Image00016.jpg" />: 6次
<img alt="" class="kindle-cn-inline-character" src="Image00002.jpg" />: 4次
<img alt="" class="kindle-cn-inline-character3" src="Image00009.jpg" />: 2次


In [6]:
def get_tag_content(content, full_tag):
    if " " in full_tag:
        tag = full_tag.split(" ")[0][1:]
    else:
        tag = full_tag[1:-1]
    pattern = re.compile(f'{full_tag}(.*?)</{tag}>', re.DOTALL)
    matches = pattern.findall(content)
    for match in matches:
        return match.strip()
    return None

def get_tag_set(content):
    pattern = re.compile(r'<(\w+)([^>]*)>')
    # 使用正则表达式查找所有匹配的标签
    matches = pattern.findall(content)
    tag_set = set()
    tag_set.add("&#13;")
    for match in matches:
        tag, attributes = match
        if tag == "img":
            continue
        full_tag = f'<{tag}{attributes}>'
        tag_set.add(full_tag)
        tag_tail = f'</{tag}>'
        tag_set.add(tag_tail)
    return tag_set

def delete_tag(content, tag_set):
    for tag in tag_set:
        content = re.sub(tag, '', content)
    return content

# print(get_tag_set(content))


In [14]:
import re

def merge_same_tags(html_string):
    # 使用正则表达式匹配所有标签及其内容
    tag_set = get_tag_set(html_string)
    pattern = r'<(\w+)([^>]*)>(.*?)</\1>'
    matches = re.findall(pattern, html_string, re.DOTALL)

    # 用于存储合并后的标签内容
    merged_tags = []
    current_tag = None
    current_content = []

    for tag, attrs, content in matches:
        # 创建一个唯一的键来标识相同的标签和属性
        key = (tag, attrs)

        # 如果当前标签与上一个标签相同，则合并内容
        if key == current_tag:
            content = delete_tag(content, tag_set)
            # print('content',content)
            current_content.append(content.strip())
        else:
            # 如果当前标签与上一个标签不同，则保存上一个标签的内容
            if current_tag is not None:
                merged_tags.append((current_tag, ' '.join(current_content)))
            # 开始新的标签内容
            current_tag = key
            content = delete_tag(content, tag_set)
            current_content = [content.strip()]

    # 添加最后一个标签的内容
    if current_tag is not None:
        merged_tags.append((current_tag, ' '.join(current_content)))

    # 构建新的 HTML 字符串
    merged_html = ''
    for (tag, attrs), content in merged_tags:
        if len(content) > 10:
            content = content.replace('\n','')
            content = content.replace(' ','')
            content = content.replace('\u3000','')
            merged_html += f'<{tag}{attrs}>{content}</{tag}>\n'
        elif len(content) == 0:
            continue
        else:
            merged_html += f'<{tag}{attrs}>{content}</{tag}>\n'

    return merged_html.strip()

# 合并相同标签的内容
merged_html = merge_same_tags(content)
print(merged_html)

<h1 id="sigil_toc_id_3">庄公</h1>
<p class="kindle-cn-para-no-indent">【题解】</p>
<p>鲁庄公名同，取名之由，是生日正巧与父亲桓公相同之故。桓公死时，子同才十三岁；由于死得意外，生前来不及考虑为儿子定亲的事。母亲文姜，又似乎把心思都花在与同父异母兄齐襄公私通上，竟把儿子的婚姻大事耽搁下来。于是鲁庄公成年以后便进行了一次自由恋爱，与任姓支族党氏之女孟任割臂为盟，私定终身。但这次自主婚姻在“奔则为妾”的时代里得不到正式承认。庄公二十一年，母文姜死；二十四年，才娶齐女哀姜为夫人，这年他已三十七岁了。这次正式婚姻并不中庄公的意，他的情爱仍在孟任身上。哀姜受到冷落之余，与庄公之弟庆父、叔牙私通，并造成了庄公身后“鲁难未已”的动乱局面。庄公在位三十二年（前693—前662年）。</p>
<p class="kindle-cn-para-no-indent">【原文】</p>
<p>3.1.1元年春王正月。公何以不言即位？《春秋》君弑，子不言即位。君弑则子何以不言即位？隐之也〔1〕。孰隐？隐子也。</p>
<p class="kindle-cn-para-no-indent">【注释】</p>
<p>〔1〕隐：伤痛。《解诂》：“隐痛是子之祸，不忍言即位。”</p>
<p class="kindle-cn-para-no-indent">【译文】</p>
<p>元年春周历正月。庄公为什么不说即位？《春秋》国君被杀，儿子不说即位。国君被杀那么儿子为什么不说即位？是伤痛他。伤痛谁？是伤痛儿子。</p>
<p class="kindle-cn-para-no-indent">【原文】</p>
<p>3.1.2三月，夫人孙于齐。孙者何？孙犹孙也〔1〕，内讳奔，谓之孙。夫人固在齐矣〔2〕，其言孙于齐何？念母也，正月以存君〔3〕，念母以首事〔4〕。夫人何以不称姜氏？贬。曷为贬？与弑公也。其与弑公奈何？夫人谮公于齐侯：“公曰：‘同非吾子，齐侯之子也〔5〕。’”齐侯怒，与之饮酒。于其出焉，使公子彭生送之〔6〕；于其乘焉〔7〕，搚干而杀之〔8〕。念母者，所善也，则曷为于其念母焉贬？不与念母也〔9〕。</p>
<p class="kindle-cn-para-no-indent">【注释】</p>
<p>〔1〕孙犹孙也：下孙读作逊

In [50]:
explanation_pattern = re.compile(r'<p class="kindle-cn-para-no-indent">【题解】</p>\s*<p>(.*?)</p>', re.DOTALL)
original_pattern = re.compile(r'<p class="kindle-cn-para-no-indent">【原文】</p>\s*<p>(.*?)</p>', re.DOTALL)
translation_pattern = re.compile(r'<p class="kindle-cn-para-no-indent">【译文】</p>\s*<p>(.*?)</p>', re.DOTALL)
notes_pattern = re.compile(r'<p class="kindle-cn-para-no-indent">【注释】</p>\s*<p>(.*?)</p>', re.DOTALL)

def parse_merged_html(merged_html, pattern):
	matches = pattern.findall(merged_html)
	result = []
	for match in matches:
		result.append(match.strip())
	return result

explanation = parse_merged_html(merged_html, explanation_pattern)
print(explanation)	
original_texts = parse_merged_html(merged_html, original_pattern)
translations = parse_merged_html(merged_html, translation_pattern)
notes = parse_merged_html(merged_html, notes_pattern)
print(original_texts)
print(len(original_texts))
print(translations)
print(len(translations))
print(notes)
print(len(notes))


['鲁庄公名同，取名之由，是生日正巧与父亲桓公相同之故。桓公死时，子同才十三岁；由于死得意外，生前来不及考虑为儿子定亲的事。母亲文姜，又似乎把心思都花在与同父异母兄齐襄公私通上，竟把儿子的婚姻大事耽搁下来。于是鲁庄公成年以后便进行了一次自由恋爱，与任姓支族党氏之女孟任割臂为盟，私定终身。但这次自主婚姻在“奔则为妾”的时代里得不到正式承认。庄公二十一年，母文姜死；二十四年，才娶齐女哀姜为夫人，这年他已三十七岁了。这次正式婚姻并不中庄公的意，他的情爱仍在孟任身上。哀姜受到冷落之余，与庄公之弟庆父、叔牙私通，并造成了庄公身后“鲁难未已”的动乱局面。庄公在位三十二年（前693—前662年）。']
['3.1.1元年春王正月。公何以不言即位？《春秋》君弑，子不言即位。君弑则子何以不言即位？隐之也〔1〕。孰隐？隐子也。', '3.1.2三月，夫人孙于齐。孙者何？孙犹孙也〔1〕，内讳奔，谓之孙。夫人固在齐矣〔2〕，其言孙于齐何？念母也，正月以存君〔3〕，念母以首事〔4〕。夫人何以不称姜氏？贬。曷为贬？与弑公也。其与弑公奈何？夫人谮公于齐侯：“公曰：‘同非吾子，齐侯之子也〔5〕。’”齐侯怒，与之饮酒。于其出焉，使公子彭生送之〔6〕；于其乘焉〔7〕，搚干而杀之〔8〕。念母者，所善也，则曷为于其念母焉贬？不与念母也〔9〕。', '3.1.3夏，单伯逆王姬〔1〕。单伯者何？吾大夫之命乎天子者也〔2〕。何以不称使？天子召而使之也。逆之者何？使我主之也〔3〕。曷为使我主之？天子嫁女乎诸侯，必使诸侯同姓者主之。诸侯嫁女于大夫，必使大夫同姓者主之。', '3.1.4秋，筑王姬之馆于外〔1〕。何以书？讥。何讥尔？筑之，礼也；于外，非礼也。于外何以非礼？筑于外，非礼也〔2〕。其筑之何以礼？主王姬者必为之改筑。主王姬者则曷为必为之改筑？于路寝则不可〔3〕，小寝则嫌〔4〕，群公子之舍则以卑矣〔5〕，其道必为之改筑者也。', '3.1.5冬十月乙亥，陈侯林卒〔1〕。王使荣叔来锡桓公命〔2〕。锡者何？赐也。命者何？加我服也〔3〕。其言桓公何？追命也。', '3.1.6王姬归于齐。何以书？我主之也。齐师迁纪郱、鄑、郚〔1〕。迁之者何？取之也。取之，则曷为不言取之也？为襄公讳也。外取邑不书，此何以书？大之也。何大尔？自是始灭也〔2〕。', '3.2.1二年春王二月，葬陈庄公。夏，公子庆父帅师伐於馀丘〔1〕。於馀

In [28]:
def parse_notes(notes):
	


'〔1〕逆：《穀梁传》同，《左传》作送。故《公羊传》、《穀梁传》皆以单伯为鲁大夫，《左传》则以单伯为周王之卿士。《通义》：“左氏经云：‘单伯送王姬。’误也。经书‘单伯会诸侯于鄄’，‘单伯如齐’，‘单伯至自齐’，并是内大夫之辞。且逆，则据往之日书，先行单伯而后筑馆可也；送，则据来之日书，尚未有以居王姬也，是不可通。”说得很有道理。单伯：单为氏，伯为字。王姬：周王室之女的通称。此王姬为将下嫁给齐襄公的；《庄公十一年》的王姬则是下嫁给齐桓公的。〔2〕吾大夫之命乎天子者也：《解诂》：“礼：诸侯三年一贡士于天子，天子命与诸侯辅助为政，所以通贤共治，示不独专，重民之至。大国举三人，次国举二人，小国举一人。”《礼记·射义》：“诸侯岁献贡士于天子，天子试之于射宫。”郑玄注：“三岁而贡士。旧说云：大国三人，次国二人，小国一人。”说同。〔3〕主：主婚。'

In [48]:
import re

# 示例文本
text = '〔1〕路寝：《解诂》：“公之正居也。”参见3.1.4注〔3〕。〔2〕正寝：《穀梁传》：“寝疾居正寝，正也。男子不绝于妇人之手，以齐终也。”后世有“寿终正寝”之语。'

# 正则表达式模式
pattern = r'〔\d+〕\s*(\w+)：'

# 使用正则表达式匹配
match = re.findall(pattern, text)

# 提取被注释的字
if match:
    for word in match:
        print(f"被注释的字是: {word}")
else:
    print("未找到匹配的注释字")

被注释的字是: 路寝
被注释的字是: 正寝


In [49]:
indexs = []
for i in range(len(match)):
    title = f"〔{i+1}〕{match[i]}"
    index = text.find(title)
    indexs.append(index)
    end = index
    
for i in range(len(indexs)):
    if i == len(indexs)-1:
        sentence = text[indexs[i]:]
    else:
        sentence = text[indexs[i]:indexs[i+1]]
    print(sentence)


〔1〕路寝：《解诂》：“公之正居也。”参见3.1.4注〔3〕。
〔2〕正寝：《穀梁传》：“寝疾居正寝，正也。男子不绝于妇人之手，以齐终也。”后世有“寿终正寝”之语。


In [46]:
indexs 

[0, 31]

In [41]:
text = text[0:31]
print(text)


〔1〕路寝：《解诂》：“公之正居也。”参见3.1.4注〔3〕。


In [32]:
import re

text = '''〔1〕路寝：《解诂》：“公之正居也。”参见3.1.4注〔3〕。〔2〕正寝：《穀梁传》：“寝疾居正寝，正也。男子不绝于妇人之手，以齐终也。”后世有“寿终正寝”之语。'''

# 正则表达式匹配每个列表项
pattern = re.compile(r'〔(\d+)〕(.+?)(?=〔\d+〕|$)', re.DOTALL)

# 查找所有匹配
matches = pattern.findall(text)

# 将匹配结果转换为列表项
items = [f"【{match[0]}】{match[1].strip()}" for match in matches]

for item in items:
    print(item)


【1】路寝：《解诂》：“公之正居也。”参见3.1.4注
【3】。
【2】正寝：《穀梁传》：“寝疾居正寝，正也。男子不绝于妇人之手，以齐终也。”后世有“寿终正寝”之语。


In [6]:
import re

def find_element_by_id(html_string, anchor):
    pattern = r'<[^>]* id="{}"[^>]*>'.format(re.escape(anchor))
    match = re.search(pattern, html_string)
    if match:
        start_pos = match.start()
        end_pos = match.end()
        matched_element = match.group(0)
        return start_pos, end_pos, matched_element
    return None

# 示例 HTML 字符串
html_string = '''
<div id="content">Some content</div>
<p asdas id="intro"asdasd>Introduction</p>
<span id="summary">Summary</span>
'''

anchor = "intro"
result = find_element_by_id(html_string, anchor)

if result:
    start_pos, end_pos, matched_element = result
    print(f"Matched element: {matched_element}")
    print(f"Start position: {start_pos}")
    print(f"End position: {end_pos}")
else:
    print("No match found")

Matched element: <p asdas id="intro"asdasd>
Start position: 38
End position: 64


In [3]:
nav_tree = extract_nav_structure(book)
# print(nav_tree)
# print(nav_tree.count_leaf_nodes())

Root: None
  总目录: text00000.html
  春秋公羊传译注: text00002.html
    书名页: text00002.html
    目录: text00003.html
    前言: text00004.html
    隐公: text00004.html#sigil_toc_id_1
    桓公: text00004.html#sigil_toc_id_2
    庄公: text00004.html#sigil_toc_id_3
    闵公: text00004.html#sigil_toc_id_4
    僖公: text00004.html#sigil_toc_id_5
    文公: text00004.html#sigil_toc_id_6
    宣公: text00004.html#sigil_toc_id_7
    成公: text00004.html#sigil_toc_id_8
    襄公: text00004.html#sigil_toc_id_9
    昭公: text00004.html#sigil_toc_id_10
    定公: text00004.html#sigil_toc_id_11
    哀公: text00004.html#sigil_toc_id_12
  孝经译注: text00006.html
    书名页: text00006.html
    目录: text00007.html
    前言: text00009.html#aa1
    译注说明: text00009.html#sigil_toc_id_1
    开宗明义章第一: text00009.html#sigil_toc_id_2
    天子章第二: text00009.html#sigil_toc_id_3
    诸侯章第三: text00009.html#sigil_toc_id_4
    卿大夫章第四: text00009.html#sigil_toc_id_5
    士章第五: text00009.html#sigil_toc_id_6
    庶人章第六: text00009.html#sigil_toc_id_7
    三才章第七: text00009.html#s

In [22]:
file_name =  "text00004.html" 
anchor_name = "sigil_toc_id_6"
def get_chapter_content(book, chapter_src, anchor=None):
	for item in book.get_items():
		if item.get_type() == ebooklib.ITEM_DOCUMENT:
			# print(item.get_name())
			if item.get_name() == file_name:
				html_content = item.get_body_content()
				html_string = html_content.decode('utf-8')
				print(html_string)
				print(len(html_string))
				break
content = get_chapter_content(book, file_name)

<div></div>&#13;
<h1>前言</h1>&#13;
<h3 class="sigil_not_in_toc">一</h3>&#13;
<p>&#13;
<span style="font-size:1.1em;">《春秋》是孔子根据鲁国史记修定的，这一点，传《春秋》的《左传》、《公羊传》、《穀梁传》说法是一致的。杨伯峻先生在《春秋左传注·前言》中说：“孔丘未曾修或作《春秋》”，只“用《鲁春秋》作过教本，传授弟子”。在这里，我们要对孔子修定《春秋》多说几句。</span>&#13;
</p>&#13;
<p>&#13;
<span style="font-size:1.1em;">《孟子》记录了孔子自己对《春秋》所说的两句话，一句见于《滕文公下》：</span>&#13;
</p>&#13;
<p style="text-indent:2em;margin-left:2em;">&#13;
<span class="kindle-cn-kai">世衰道微，邪说暴行有作，臣弑其君者有之，子弑其父者有之，孔子惧，作《春秋》。《春秋》，天子之事也。是故孔子曰：“知我者其惟《春秋》乎，罪我者其唯《春秋》乎！”……孔子成《春秋》而乱臣贼子惧。</span>&#13;
</p>&#13;
<p class="kindle-cn-para-no-indent">&#13;
<span style="font-size:1.1em;">一句见于《离娄下》：</span>&#13;
</p>&#13;
<p style="text-indent:2em;margin-left:2em;">&#13;
<span class="kindle-cn-kai">王者之迹熄而《诗》亡，《诗》亡然后《春秋》作。……其事则齐桓、晋文，其文则史，孔子曰：“其义则丘窃取之矣。”</span>&#13;
</p>&#13;
<p class="kindle-cn-para-no-indent">&#13;
<span style="font-size:1.1em;">可见孔子把《春秋》看成是表现自己政治观点的著作，《春秋》之“文”虽是编年史，可是它还有“义”，那就是“王者”之道。</span>&#13;
</p>&#13;
<p>&#13;
<span style="font-size:

In [70]:
text = text.replace('\n','')
text = text.replace('&#13;','')
result = re.sub(r'<span class="math-super">.*?</span>', '', text)

# 使用正则表达式去掉 <span style="font-size:0.9em;"> 和 </span>
result = re.sub(r'<span style="font-size:0.9em;">|</span>|<p>|</p>|<p class="kindle-cn-para-no-indent">|<span style="font-size:1.2em;">|<span style="font-size:0.8em;">', '', result)

result = result.replace('【原文】','\n【原文】\n')
result = result.replace('【译文】','\n【译文】\n')
result = result.replace('【注释】','\n【注释】\n')
result = result.replace(' ','')

print(result)  # 输出: "1.1.2　三月，公及邾娄仪父盟于眜。"


【原文】
1.1.2　三月，公及邾娄仪父盟于眜。及者何？与也。会、及、暨皆与也，曷为或言会，或言及，或言暨？会，犹最也；及，犹汲汲也；暨，犹暨暨也。及，我欲之；暨，不得已也。仪父者何？邾娄之君也。何以名？字也。曷为称字？褒之也。曷为褒之？为其与公盟也。与公盟者众矣，曷为独褒乎此？因其可褒而褒之。此其为可褒奈何？渐进也。眜者何？地期也。
【注释】
〔1〕邾娄：鲁国附近的小国，《左传》、《穀梁传》都作“邾”（后文同），《礼记·檀弓》与《公羊传》同，《国语》、《晏子春秋》、《孟子》、《战国策》、《韩非子》等都作“邹”（为“邾娄”之急读），《吕氏春秋》或作“邾”，或作“邹”，所指皆同。仪父，名克，谥庄公，邾娄国君。　眜：鲁国地名，《穀梁传》同，《左传》作“蔑”，古音同。《竹书纪年·周平王四十九年》作“姑蔑”。姑蔑或姑眜当为正称，《春秋》因隐公名姑息而讳省“姑”字。王献唐《三邾附近国邑考》：“今泗水卞城南有姑蔑城……故城在（山东省）泗水县东五十里。”〔2〕最：聚。《礼记·乐记》：“会以聚众。”郑玄注：“会，犹聚也。聚或为最。”《史记·殷本纪》：“大最乐戏于沙丘。”《集解》引徐广曰：“最一作聚。”可见古最、聚可互通。〔3〕汲汲：心情急切的样子。《礼记·问丧》：“汲汲然，如有追而弗及也。”孔颖达疏：“汲汲然者，促急之情也。”〔4〕暨暨：神态果毅的样子。《礼记·玉藻》：“戎容暨暨。”注：“果毅貌也。”这里解释会、及、暨三个同义词的区别。及、暨在语感上的特有色彩，现在已经不存在了。〔5〕名：用作动词。〔6〕字：表字。《礼记·檀弓上》：“幼名，冠字。”疏：“始生三月而加名……人年二十，有为人父之道，朋友等类不可复呼其名，故冠而加字。”称人表字是对人尊敬的一种表示。董仲舒《春秋繁露·爵国》节引《公羊传·庄公十年》文而发挥说：“氏不若人，人不若名，名不若字：凡四等，命曰附庸，三代共之。”意思是说，《春秋》对附庸小国的称呼有四个等级，从上到下分别是称字、称名、称人、称氏。〔7〕曷为称字？褒之也：《春秋》常用称呼来褒贬。《公羊传·桓公十五年》：“邾娄人、牟人、葛人来朝。皆何以称人？夷狄之也。”可见对邾娄称人是一种贬称。而这里称字，传又说是“褒之”。所以《通义》说：“进退相较，明仪父本在名等。”按照邾娄国的地位，其国君本来只列在以名称呼的一等，现在用字称呼他，所以寓有褒意。〔8〕为其与公盟也：

In [54]:
# print(content[20820:85391])
import re

# 定义正则表达式模式
pattern = re.compile(r'<(\w+)([^>]*)>')

# 使用正则表达式查找所有匹配的标签
matches = pattern.findall(content[20820:85391])

# 统计标签及其出现次数
tag_count = {}
for match in matches:
    tag, attributes = match
    full_tag = f'<{tag}{attributes}>'
    if full_tag in tag_count:
        tag_count[full_tag] += 1
    else:
        tag_count[full_tag] = 1

# 输出统计结果
for tag, count in tag_count.items():
    if count > 1:
        print(f'{tag}: {count}次')

<p class="kindle-cn-para-no-indent">: 148次
<p>: 470次
<span style="font-size:0.8em;">: 126次
<span style="font-size:0.9em;">: 73次
<span class="math-super">: 221次
<span style="font-size:1.2em;">: 50次
<img alt="" class="kindle-cn-inline-character3" src="Image00005.jpg" />: 2次
<img alt="" class="kindle-cn-inline-character" src="Image00006.jpg" />: 2次
<img alt="" class="kindle-cn-inline-character1" src="Image00006.jpg" />: 2次
<img alt="" class="kindle-cn-inline-character4" src="Image00006.jpg" />: 3次
<img alt="" class="kindle-cn-inline-character3" src="Image00009.jpg" />: 2次
<img alt="" class="kindle-cn-inline-character3" src="Image00010.jpg" />: 2次


In [21]:
import re
pattern = re.compile(r'<p>(.*?)</p>', re.DOTALL)

# 使用正则表达式查找所有匹配的内容
matches = pattern.findall(content[20820:85391])

# 输出匹配的内容
for match in matches:
    print(match.strip())


&#13;
<span style="font-size:0.8em;">鲁隐公名息姑（据《世本》和《史记·十二诸侯年表》，《史记·鲁周公世家》作“息”，脱“姑”字），是惠公（名弗湟，公元前768—723年在位）的庶长子。从鲁国的第一个国君伯禽（周公的长子）算起，他是第七世孙，第十三位国君。据《左传》，惠公娶宋国女孟子为元配夫人，无子；孟子早死，媵妾声子作为继室，生下隐公；宋武公有个女儿仲子，天生手纹有点像“鲁夫人”三字，所以也嫁给了惠公，生下桓公；不久，惠公就死了。《史记·鲁周公世家》则说，声子生下儿子息，息长大后，惠公为他娶宋女，把宋女迎来，惠公发现她十分美丽，就把她夺下做了自己的妻子，生下允（桓公的名字）；惠公去世时，允还年幼。《史记》所说的宋女，当然就是《左传》的仲子，不过它插进的故事，是《左传》所没有的。《左传·隐公元年》和《史记·鲁周公世家》都说惠公在死之前就立桓公为太子，惠公死后，因桓公年少，由隐公摄政，这是和《公羊传》不同的地方。鲁隐公在位11年（前722—前712年）。</span>&#13;
&#13;
<span style="font-size:0.9em;">1.1.1　元年春王正月<span class="math-super">〔1〕</span>&#13;
 。</span>&#13;
&#13;
<span style="font-size:1.2em;">元年者何？君之始年也<span class="math-super">〔2〕</span>&#13;
 。春者何？岁之始也。王者孰谓？谓文王也<span class="math-super">〔3〕</span>&#13;
 。曷为先言王而后言正月？王正月也。何言乎王正月？大一统也<span class="math-super">〔4〕</span>&#13;
 。公何以不言即位？成公意也。何成乎公之意？公将平国而反之桓<span class="math-super">〔5〕</span>&#13;
 。曷为反之桓？桓幼而贵，隐长而卑。其为尊卑也微，国人莫知<span class="math-super">〔6〕</span>&#13;
 。隐长又贤，诸大夫扳隐而立之<span class="math-super">〔7〕</span>&#13;
 。隐于是焉而辞立<span c

In [18]:
import re
pattern = re.compile(r'<span style="font-size:1.2em;">(.*?)</span>', re.DOTALL)

# 使用正则表达式查找所有匹配的内容
matches = pattern.findall(content[20820:85391])

# 输出匹配的内容
for match in matches:
    print(match.strip())


元年者何？君之始年也<span class="math-super">〔2〕
及者何？与也。会、及、暨皆与也，曷为或言会，或言及，或言暨？会，犹最也<span class="math-super">〔2〕
克之者何？杀之也<span class="math-super">〔2〕
宰者何<span class="math-super">〔2〕
孰及之？内之微者也<span class="math-super">〔2〕
祭伯者何？天子之大夫也。何以不称使？奔也<span class="math-super">〔2〕
何以不日<span class="math-super">〔5〕
入者何？得而不居也<span class="math-super">〔3〕
无骇者何？展无骇也<span class="math-super">〔5〕
纪履<img alt="" class="kindle-cn-inline-character" src="Image00006.jpg" />&#13;
 者何？纪大夫也。何以不称使？婚礼不称主人<span class="math-super">〔3〕
伯姬者何？内女也<span class="math-super">〔1〕
纪子伯者何？无闻焉尔。
夫人子氏者何？隐公之母也<span class="math-super">〔2〕
何以书？记异也。日食，则曷为或日，或不日，或言朔，或不言朔？曰某月某日朔日有食之者，食正朔也。其或日，或不日，或失之前，或失之后<span class="math-super">〔2〕
何以不书葬？天子记崩不记葬，必其时也<span class="math-super">〔2〕
尹氏者何？天子之大夫也。其称尹氏何？贬。曷为贬？讥世卿<span class="math-super">〔2〕
武氏子者何？天子之大夫也。其称武氏子何？讥。何讥尔？父卒，子未命也<span class="math-super">〔2〕
葬者曷为或日或不日？不及时而日<span class="math-super">〔2〕
牟娄者何？杞之邑也。外取邑不书，此何以书？疾始取邑也。
曷为以国氏<span class="math-super">〔3〕
遇者何？不期也<span class="math-super">〔2〕
翚者何？公子翚也。何以不称公

In [20]:
import re
pattern = re.compile(r'<span style="font-size:0.9em;">(.*?)</span>', re.DOTALL)

# 使用正则表达式查找所有匹配的内容
matches = pattern.findall(content[20820:85391])

# 输出匹配的内容
for match in matches:
    print(match.strip())
    # break

1.1.1　元年春王正月<span class="math-super">〔1〕
1.1.2　三月，公及邾娄仪父盟于眜<span class="math-super">〔1〕
1.1.3　夏五月，郑伯克段于鄢<span class="math-super">〔1〕
1.1.4　秋七月，天王使宰咺来归惠公、仲子之赗<span class="math-super">〔1〕
1.1.5　九月，及宋人盟于宿<span class="math-super">〔1〕
1.1.6　冬十有二月，祭伯来<span class="math-super">〔1〕
公子益师卒<span class="math-super">〔4〕
1.2.1　二年春，公会戎于潜<span class="math-super">〔1〕
夏五月，莒人入向<span class="math-super">〔2〕
无骇帅师入极<span class="math-super">〔4〕
1.2.2　秋八月庚辰，公及戎盟于唐<span class="math-super">〔1〕
九月，纪履<img alt="" class="kindle-cn-inline-character" src="Image00006.jpg" />&#13;
 来逆女<span class="math-super">〔2〕
1.2.3　冬十月，伯姬归于纪。
纪子伯、莒子盟于密<span class="math-super">〔3〕
1.2.4　十有二月乙卯，夫人子氏薨<span class="math-super">〔1〕
郑人伐卫<span class="math-super">〔3〕
1.3.1　三年春王二月己巳，日有食之<span class="math-super">〔1〕
1.3.2　三月庚戌，天王崩<span class="math-super">〔1〕
1.3.3　夏四月辛卯，尹氏卒<span class="math-super">〔1〕
1.3.4　秋，武氏子来求赙<span class="math-super">〔1〕
1.3.5　八月庚辰，宋公和卒<span class="math-super">〔1〕
1.3.6　冬十有二月，齐侯、郑伯盟于石门<span class="math-super">〔1〕
1.4.1　四年

In [25]:
pattern = r'<[^>]+ id="sigil_toc_id_3">'
match = re.search(pattern, content)

if match:
    position = match.start()
    print(f"The string with id 'sigil_toc_id_1' is found at position: {position}")
else:
    print("The string with id 'sigil_toc_id_1' was not found.")

The string with id 'sigil_toc_id_1' is found at position: 151169


In [7]:
print(nav_structure)

总目录: text00000.html
春秋公羊传译注: text00002.html
  书名页: text00002.html
  目录: text00003.html
  前言: text00004.html
  隐公: text00004.html#sigil_toc_id_1
  桓公: text00004.html#sigil_toc_id_2
  庄公: text00004.html#sigil_toc_id_3
  闵公: text00004.html#sigil_toc_id_4
  僖公: text00004.html#sigil_toc_id_5
  文公: text00004.html#sigil_toc_id_6
  宣公: text00004.html#sigil_toc_id_7
  成公: text00004.html#sigil_toc_id_8
  襄公: text00004.html#sigil_toc_id_9
  昭公: text00004.html#sigil_toc_id_10
  定公: text00004.html#sigil_toc_id_11
  哀公: text00004.html#sigil_toc_id_12
孝经译注: text00006.html
  书名页: text00006.html
  目录: text00007.html
  前言: text00009.html#aa1
  译注说明: text00009.html#sigil_toc_id_1
  开宗明义章第一: text00009.html#sigil_toc_id_2
  天子章第二: text00009.html#sigil_toc_id_3
  诸侯章第三: text00009.html#sigil_toc_id_4
  卿大夫章第四: text00009.html#sigil_toc_id_5
  士章第五: text00009.html#sigil_toc_id_6
  庶人章第六: text00009.html#sigil_toc_id_7
  三才章第七: text00009.html#sigil_toc_id_8
  孝治章第八: text00009.html#sigil_toc_id_9
  圣治章第九: text000